In [1]:
# Import necessary libraries
import requests as req
import sys
import os
import numpy as np
import pandas as pd
import shapely.geometry as geom
import rasterio as rs
from rasterio.mask import mask as rs_mask
#import matplotlib.pyplot as plt
import plotly.graph_objects as go
import timeit

from utils.Address import Address
from utils.Building import Building
#from utils import Building as blds

In [2]:
# File paths for metadata for DSM and DTM datasets
# File paths for the raw and processed metadata csv files
metadata_folder_path = os.path.join(os.getcwd(), 'data', 'metadata')
   
# Preprocessed and merge csv file path
metadata_processed_merged_file_path = os.path.join(metadata_folder_path, 
                                                   'GeoTIFF_1m_metadata_processed.csv')

In [3]:
# Create the building instance from a valid address in Flanders
while True:
    input_address = 'Hofstraat 37, 2910 Essen'
    input_address = 'Elzendreef 14, 2910 Essen'
    input_address = 'Schoenmarkt 35, 2000 Antwerpen'
    # input_address = 'dsfd'
    # input_address = 'Elzendreef 14'
    
    # Get address as user input
    # input_address = input()
    if input_address == 'quit':  # exit the loop to quit
        break
    # Attempt initializing the Building object 
    building_obj = Building(input_address)
    
    # Display suggestions if there are multiple matching addresses.
    if len(building_obj.suggestions["SuggestionResult"])>1:
        print("Did you mean one of the following addresses?:")
        print(*building_obj.suggestions['SuggestionResult'], sep="\n")
    
    # Exit the loop if a valid address is given
    if building_obj.valid_address_conditional == True:
        print("Address found:")
        print(building_obj)
        break

Address found:
Schoenmarkt 35, 2000 Antwerpen


In [7]:
class CHM():
    """
    A class to represent surface of a building
    ...
    Attributes
    ----------
     selected_building
        An instance of class Building representing a building in Flanders

    Methods
    -------
    dilate_building_polygon
    get_masked_raster
    get_CHM
    
    Methods
    -------
    xx
        yyy
    """

    dem_data_types = ['DSM', 'DTM']
    
    def __init__(self, selected_building, use_local_copy=False ):
        """
        Initializes an instance of the class CHM (Canopy Height Model)
        :param selected_building: An instance of class Building
        """ 
        # Assign the building object
        self.building = selected_building
        
        # Dilate the building polygon
        dilation = 5  # in meters
        self.dilate_building_polygon()
        
        # Load masked data for both DSM and DTM
        self.file_links = []
        self.masked_rasters = []
        self.masked_transforms = []
        self.nodata_values = []
        for dem_data_type in self.dem_data_types:
            file_link = self.building.metadata_building[dem_data_type.lower()+"_file_link"].values[0]
            # If use_local_copy is True, a local copy of the zip file can be used
            
            if use_local_copy:
                # file_link = self.get_local_zip_file_path(file_link)
                if dem_data_type == 'DSM':
                    file_link = 'zip+C:\\Users\\ecebo\\MyRepos\\3D_houses\\data\\dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k15.zip!/GeoTIFF/DHMVIIDSMRAS1m_k15.tif'
                else:
                    file_link = 'zip+C:\\Users\\ecebo\\MyRepos\\3D_houses\\data\\dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDTMRAS1m_k15.zip!/GeoTIFF/DHMVIIDTMRAS1m_k15.tif'
    
            self.file_links.append(file_link)
            masked_raster, masked_transform, nodata_value = self.get_masked_raster(file_link)
            self.masked_rasters.append(masked_raster)
            self.masked_transforms.append(masked_transform)
            self.nodata_values.append(nodata_value)
        
        # Calculate the CHM (Canopy Height Model) from DSM and DTM data
        self.get_CHM()
        
        # Calculate the height of the building
        self.height = np.nanmax(self.masked_raster_chm)

    def __str__(self):
        """
        Prints the address
        """
        return f"{self.building.address}, maximum height: {self.height}"
    
    def dilate_building_polygon(self, dilation=2):
        
        self.dilated_polygon = self.building.building_polygon.buffer(dilation)

    
    def get_masked_raster(self, file_link):
        
        # GeoTIFF file path
        file_path = file_link
        
        # Polygon for masking
        dilated_polygon = self.dilated_polygon

        # Access the TIFF file
        try:
            with rs.open(file_path) as src:
                # Mask the file with the dilated polygon
                nodata_value = src.nodata
                masked_raster, masked_transform = rs_mask(src, [dilated_polygon], 
                                                          all_touched=True, nodata = nodata_value, 
                                                          filled=True, crop=True, 
                                                          pad=True, indexes=1)
        except:
            masked_raster, masked_transform = np.ndarray(0), np.ndarray(0)
            nodata_value = np.nan
        
        return masked_raster, masked_transform, nodata_value
    
    def get_local_zip_file_path(self, file_link):
        
        local_file_link = ''
        return local_file_link
    
    def get_CHM(self):
        
        # Subtract the DSM data from DTM
        self.masked_raster_chm = self.masked_rasters[0] - self.masked_rasters[1]
        # Set values out of the mask to numpy nan
        mask_nodata = self.masked_rasters[0]==self.nodata_values[0]
        self.masked_raster_chm[mask_nodata] = np.nan
        
    def plot_CHM_3d(self):
        
        z_data = self.masked_raster_chm
        x, y = np.arange(z_data.shape[1]),np.arange(z_data.shape[0])
        fig = go.Figure(data=[go.Surface(z=z_data, x=x, y=y)])
        fig.update_layout(title='a', autosize=True)
        fig.show()
        
        return fig
        

In [ ]:
tStart = timeit.default_timer()
building_CHM_model = CHM(building_obj, use_local_copy=False)
# building_CHM_model.__dict__
tStop = timeit.default_timer()
print(tStop-tStart)

In [ ]:
building_CHM_model.file_links[0]

In [68]:
building_CHM_model.file_links[0]

'zip+C:\\Users\\ecebo\\MyRepos\\3D_houses\\data\\dhm-vlaanderen-ii-dsm-raster-1m/DHMVIIDSMRAS1m_k15.zip!/GeoTIFF/DHMVIIDSMRAS1m_k15.tif'

In [50]:
# z_data = masked_raster
z_data = building_CHM_model.masked_raster_chm
x, y = np.arange(z_data.shape[1]),np.arange(z_data.shape[0])
fig = go.Figure(data=[go.Surface(z=z_data, x=x, y=y)])
fig.update_layout(title=print(building_CHM_model.building), autosize=True)
fig.show()

Schoenmarkt 35, 2000 Antwerpen


In [52]:
fig.write_image('temp.svg')

ValueError: Failed to start Kaleido subprocess. Error stream:

& was unexpected at this time.
